# Instalación y datos

## Instalación de postgres y carga de datos

In [35]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# pin package versions for google colab compatibility
!pip install SQLAlchemy==1.4.46
!pip install ipython-sql==0.4.1
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
%sql create schema control2;
%sql set search_path to control2;

from google.colab import output
output.clear()

### Airports

In [36]:
%%sql

CREATE TABLE airports(
    id bigint,
    name varchar(128),
    city varchar(128),
    country varchar(64),
    iata_code varchar(3),
    icao_code varchar(4),
    latitude decimal(9, 6),
    longitude decimal(9, 6),
    altitude float,
    timezone decimal(4, 1),
    dst varchar(1),
    tz varchar(64),
    airport_type varchar(8),
    source varchar(16)
);

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.DuplicateTable) relation "airports" already exists

[SQL: CREATE TABLE airports(
    id bigint,
    name varchar(128),
    city varchar(128),
    country varchar(64),
    iata_code varchar(3),
    icao_code varchar(4),
    latitude decimal(9, 6),
    longitude decimal(9, 6),
    altitude float,
    timezone decimal(4, 1),
    dst varchar(1),
    tz varchar(64),
    airport_type varchar(8),
    source varchar(16)
);]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [37]:
!wget https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat

--2023-04-29 03:29:11--  https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1127225 (1.1M) [text/plain]
Saving to: ‘airports.dat.8’

airports.dat.8      100%[===================>]   1.07M  --.-KB/s    in 0.05s   

2023-04-29 03:29:11 (22.1 MB/s) - ‘airports.dat.8’ saved [1127225/1127225]



In [38]:
%%sql

COPY airports FROM '/content/airports.dat' WITH (FORMAT CSV, DELIMITER(','), NULL '\N' )

 * postgresql+psycopg2://@/postgres


""


### Airlines

In [39]:
%%sql

CREATE TABLE airlines(
    id bigint,
    name varchar(128),
    alias varchar(64),
    iata_code varchar(4),
    icao_code varchar(8),
    callsign varchar(128),
    country varchar(64),
    active varchar(1)
);

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.DuplicateTable) relation "airlines" already exists

[SQL: CREATE TABLE airlines(
    id bigint,
    name varchar(128),
    alias varchar(64),
    iata_code varchar(4),
    icao_code varchar(8),
    callsign varchar(128),
    country varchar(64),
    active varchar(1)
);]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [40]:
!wget https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat

--2023-04-29 03:29:11--  https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 396896 (388K) [text/plain]
Saving to: ‘airlines.dat.8’

airlines.dat.8      100%[===================>] 387.59K  --.-KB/s    in 0.04s   

2023-04-29 03:29:11 (10.7 MB/s) - ‘airlines.dat.8’ saved [396896/396896]



In [41]:
%%sql

COPY airlines FROM '/content/airlines.dat' WITH (FORMAT CSV, DELIMITER(','), NULL '\N' )

 * postgresql+psycopg2://@/postgres


""


### Routes

In [42]:
%%sql

CREATE TABLE routes(
    airline_code varchar(3),
    airline_id bigint,
    source_airport_code varchar(4),
    source_airport_id bigint,
    destination_airport_code varchar(4),
    destination_airport_id bigint,
    vcodeshare varchar(1),
    codeshare boolean,
    stops int,
    equipment varchar(128)
);

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.DuplicateTable) relation "routes" already exists

[SQL: CREATE TABLE routes(
    airline_code varchar(3),
    airline_id bigint,
    source_airport_code varchar(4),
    source_airport_id bigint,
    destination_airport_code varchar(4),
    destination_airport_id bigint,
    vcodeshare varchar(1),
    codeshare boolean,
    stops int,
    equipment varchar(128)
);]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [43]:
!wget https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat

--2023-04-29 03:29:12--  https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2377148 (2.3M) [text/plain]
Saving to: ‘routes.dat.8’

routes.dat.8        100%[===================>]   2.27M  --.-KB/s    in 0.06s   

2023-04-29 03:29:12 (36.1 MB/s) - ‘routes.dat.8’ saved [2377148/2377148]



In [44]:
%%sql

COPY routes(
    airline_code,
    airline_id,
    source_airport_code,
    source_airport_id,
    destination_airport_code,
    destination_airport_id,
    vcodeshare,
    stops,
    equipment
) FROM '/content/routes.dat' WITH (FORMAT CSV, DELIMITER(','), NULL '\N' );

UPDATE routes
SET codeshare = FALSE
WHERE vcodeshare = '';

UPDATE routes
SET codeshare = TRUE
WHERE vcodeshare = 'Y';

ALTER TABLE routes DROP COLUMN vcodeshare;

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.UndefinedColumn) column "vcodeshare" of relation "routes" does not exist

[SQL: COPY routes(
    airline_code,
    airline_id,
    source_airport_code,
    source_airport_id,
    destination_airport_code,
    destination_airport_id,
    vcodeshare,
    stops,
    equipment
) FROM '/content/routes.dat' WITH (FORMAT CSV, DELIMITER(','), NULL '\N' );]
(Background on this error at: https://sqlalche.me/e/14/f405)


# Esquema

## Esquema

La base de datos tiene 3 tablas: `airlines`, `airports` y `routes`. Puedes revisar el esquema en detalle [en esta página](https://openflights.org/data.html) de dónde sacamos los datos.

* `airlines(id, name, alias, iata_code, icao_code, callsign, country, active)`
* `airports(id, name, city, country, iata_code, icao_code, latitude, longitude, altitude, timezone, dst, tz, airport_type, source)`
* `routes(airline_code, airline_id, source_airport_code, source_airport_id, destination_airport_code, destination_airport_id, codeshare, stops, equipment)`

# Preguntas

Haga consultas en SQL para obtener la siguiente información:

### 1. Indique los pares de aeropuertos con más vuelos de uno al otro


In [45]:
%%sql
SELECT routes.source_airport_code, routes.destination_airport_code, COUNT(*) --en una issue lei que aceptaban seleccionar los codigos (no necesariamente los nombres) de cada aeropuerto
AS vuelos
FROM routes
GROuP BY routes.source_airport_code, routes.destination_airport_code
ORDER BY vuelos DESC
LIMIT 7;


 * postgresql+psycopg2://@/postgres


,source_airport_code,destination_airport_code,vuelos
0,ORD,ATL,20
1,ATL,ORD,19
2,ORD,MSY,13
3,HKT,BKK,13
4,MIA,ATL,12
5,ATL,MIA,12
6,AUH,MCT,12


### 2. Enumere todos las ciudades que conectan Santiago con Frankfurt


In [46]:
%%sql
SELECT DISTINCT airports.city
FROM airports
WHERE airports.id IN (
  SELECT routes.source_airport_id --#seleccione todas los aeropuertos que eran llegada de rutas que salian de santiago y a la vez fueran salida de vuelos que llegaban a frankfurt#
  FROM routes, airports
  WHERE routes.destination_airport_id = airports.id
  AND airports.city = 'Frankfurt'
) AND airports.id IN (
  SELECT routes.destination_airport_id
  FROM routes, airports
  WHERE routes.source_airport_id = airports.id
  AND airports.city = 'Santiago'
);



 * postgresql+psycopg2://@/postgres


,city
0,Alicante
1,Arrecife
2,Atlanta
3,Barcelona
4,Bilbao
5,Bogota
6,Boston
7,Brussels
8,Buenos Aires
9,Cancun


### 3. ¿Cuál es son los nombres de las aerolineas más grande? (La que tiene la mayor cantidad de rutas). Liste las 10 más grandes.

In [47]:
%%sql
SELECT airlines.name, COUNT(*) AS cantidad_rutas
FROM airlines, routes
WHERE airlines.id = routes.airline_id
GROUP BY airlines.name
ORDER BY cantidad_rutas DESC
LIMIT 10;

 * postgresql+psycopg2://@/postgres


,name,cantidad_rutas
0,Ryanair,22356
1,American Airlines,21186
2,United Airlines,19620
3,Delta Air Lines,17829
4,US Airways,17640
5,China Southern Airlines,13086
6,China Eastern Airlines,11367
7,Air China,11340
8,Southwest Airlines,10314
9,easyJet,10170


### 4. ¿Cuáles son los 5 aeropuertos más importantes? (Aeropuertos de los cuáles llegan y salen más rutas)

In [ ]:
%%sql
SELECT airports.name, COUNT(*) AS flujos
FROM airports, routes
WHERE routes.destination_airport_id = airports.id --seleccione las rutas que eran destino o llegada
OR routes.source_airport_id = airports.id 
GROUP BY airports.name
ORDER BY flujos DESC
LIMIT 5;


 * postgresql+psycopg2://@/postgres


### 5. Entregue todas las rutas (sin repetición) que van desde New York hacia Berlin y pasan por exactamente tres aeropuertos. Para esto entregue tres columnas con los códigos de cada uno de los aeropuertos involucrados.

### 6. Entregue los 10 aeropuertos más cercanos al aeropuerto 'Pdte. Carlos Ibañez del Campo Airport' (sin incluirlo), ordenados en distancia de forma ascendente.

Para calcular la distancia, puede utilizar la siguiente fórmula:

$$
distancia(p1, p2) = \sqrt{(p2.lat - p1.lat)^{2} + (p2.long - p1.long)^{2}}
$$

Es importante mencionar que esta fórmula no es precisa, pues no considera la curvatura de la tierra 😉

In [18]:
%%sql
SELECT DISTINCT airports.name,
    SQRT(POWER(airports.latitude - a1.latitude, 2) + POWER(airports.longitude - a1.longitude, 2)) AS hipotenusa
FROM airports AS a1, airports --Cruze dos copias diferentes del mismo esquema poniendo la condicion de que una de ellas fuera 'Pdte. Carlos Ibañez del Campo Airport' y calcule la hipotenusa entre ellas.
WHERE a1.name = 'Pdte. Carlos Ibañez del Campo Airport' AND a1.name <> airports.name
ORDER BY hipotenusa ASC
LIMIT 10;
 

 * postgresql+psycopg2://@/postgres


,name,distance
0,Capitan Fuentes Martinez Airport Airport,0.59135722334643043
1,Piloto Civil N. Fernández Airport,2.0785009455867467
2,Tte. Julio Gallardo Airport,2.1385578357867715
3,San Sebastián Airport,2.2269024818365981
4,El Calafate Airport,2.9744470449493634
5,Lago Argentino Airport,3.0088951271204186
6,Malvinas Argentinas Airport,3.1520820816731597
7,Hermes Quijada International Airport,3.2004746115545113
8,Santa Cruz Airport,3.7542303823293796
9,Guardiamarina Zañartu Airport,3.7604566853259991
